Load the war/abort witch captions dataset

In [ ]:
import pandas as pd
df = pd.read_csv("abortion_war_captions.csv")
df.head()

Installing VADER lexicon

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download VADER if not already installed
nltk.download('vader_lexicon')

Applying VADER

In [ ]:
# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Score from -1 to 1, where -1 is the most negative and 1 the most positive
df['sentiment_score'] = df['captions'].astype(str).apply(lambda text: sia.polarity_scores(text)['compound'])

In [ ]:
print(df[['captions', 'sentiment_score']].head())

Translation of the scores into a label

In [ ]:
def categorize_sentiment(score):
    if score <= -0.75:
        return "Very Negative"
    elif score <= -0.4:
        return "Negative"
    elif score < 0.1:
        return "Slightly Negative"
    elif score < 0.09:
        return "Neutral"
    elif score < 0.4:
        return "Slightly Positive"
    elif score < 0.75:
        return "Positive"
    else:
        return "Very Positive"

In [ ]:
df['sentiment_category'] = df['sentiment_score'].apply(categorize_sentiment)


Getting subjectivity of the content

In [ ]:
from textblob import TextBlob

# Function to get subjectivity
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
df['subjectivity'] = df['captions'].astype(str).apply(get_subjectivity)

In [ ]:
print(df[['captions', 'sentiment_score', 'subjectivity']].head())

Export to CSV file

In [ ]:
df.to_csv("sentiment_subjectivity.csv", index=False)